In [2]:
!pip install -q scipy==1.2.0
!pip install -q numpy
!pip install -q rawpy
!pip install -q Pillow

     |████████████████████████████████| 26.6MB 111kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 1.6MB 2.8MB/s 


In [0]:
!pip uninstall scipy
!pip install scipy==1.2.0

In [0]:
!python download_models.py

In [0]:
!python train_Sony_mine.py

In [0]:
!ls

In [4]:
!pwd

/content/drive/My Drive/Bakalarka-Peter_Sipos/Implementation/Learning-to-See-in-the-Dark


In [1]:
%cd /content/drive/My Drive/Bakalarka-Peter_Sipos/Implementation/Learning-to-See-in-the-Dark

/content/drive/My Drive/Bakalarka-Peter_Sipos/Implementation/Learning-to-See-in-the-Dark


In [0]:
a = []
while(1):
    a.append('1')

In [0]:
import sys
print("Python version")
print (sys.version)